In [1]:
!nvidia-smi

Fri Apr 12 16:26:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 37.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [6]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Given these locations:\n
  {query}\n\n"
  Generate a good and coherent way to spend the day during a trip.\n"
  Your plan should follow this format:\n
  Day Plan:\n
        Morning: [Activity]\n
        Mid-Morning: [Activity]\n
        Midday: [Activity]\n
        Afternoon: [Activity]\n
        Evening: [Activity]\n
        Night: [Activity]\n
        \n
        Activities can include:\n
        - Exploring [Location]\n
        - Trying local cuisine at [Restaurant]\n
        - Visiting [Landmark]\n
        - Enjoying [Recreational Activity] at [Park/Beach]\n
        - Relaxing with [Activity] at [Spa/Hotel]\n
        - Participating in [Event/Activity] at [Venue]\n
        - Shopping at [Market/Mall]\n
        - Taking a guided tour of [Attraction]\n
        - Engaging in outdoor activities like [Activity] at [Outdoor Location]\n
        - Experiencing cultural immersion at [Museum/Cultural Site]\n
        - Enjoying scenic views at [Scenic Spot]\n
        \n
        "Feel free to customize the activities based on your preferences and interests."
  <end_of_turn>\n<start_of_turn>model

  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [8]:
query = """La Pampa is a restaurant located in Beni Khiar, offering a variety of dining options. It has received a 4.4 rating from customers and is open from 7am to 1pm and 5pm to 2am daily. The restaurant's featured image showcases a vibrant and inviting ambiance, and it is located at the coordinates (36.454387640549, 10.785850418423).
Located in Beni Khiar, this bakery has earned a 4.4-star rating from 17 reviews. Its exact location is at FQ5M+V7J, and it can be contacted at 92 422 215 for inquiries or orders.
Cafe des Arcades, located at 15 Av. 14 Janvier 2011, Beni Khiar, is a cafe offering a relaxing atmosphere with a 4.4 rating based on 8 reviews. It operates daily from 4:30 AM to midnight and can be reached via phone at 90 443 842.
LA BAMBA is a Pizzeria located in beni khiar. Its rating is 4.7/5 based on 10 reviews.
Fast Food Haythem is a fast food restaurant located in Beni Khiar, Tunisia. It has a rating of 5.0 based on 4 reviews. It is located at 36.4724714, 10.7885366.
Plage Beni Khiar is a beautiful beach located in Beni Khiar, Tunisia. It boasts a 4.1 rating based on 57 reviews, and is known for its clean sands and crystal-clear waters. Visitors can enjoy swimming, sunbathing, and taking in the stunning coastal views. The beach is easily accessible via road and is an ideal destination for a relaxing day trip or beach vacation.
Located at coordinates (36.4542877, 10.7969189) in the city of Beni Khiar, Port de Beni Khiar is a maritime port renowned for its exceptional services. Despite lacking a website or featured image, the port maintains an impressive rating of 4.6 based on 9 reviews.
Located in Beni Khiar (coordinates: 45.19834, 0.35948), Restaurant la voix de la mer specializes in seafood. With a rating of 3.8 based on 4 reviews, it offers an authentic dining experience that focuses on the freshest catches of the day.
Jinen Beni Khiar is a renowned chambre d'hôtes located in Beni Khiar, Tunisia. It boasts an impressive rating of 4.7 stars from 3 reviews, showcasing its exceptional quality. Nestled at coordinates (36.469386365975, 10.807021723363), the chambre d'hôtes offers guests a convenient location for exploring the area.
Gb@ lbndq bny khyr is a 4.5-rated campsite located in Beni Khiar, Tunisia. It's a great place to relax and enjoy the outdoors. The campsite is located at coordinates (36.4724714, 10.7885366).
Pepiniere Maamoura is located in Beni Khiar, Tunisia, at (36.46499, 10.80612). It offers a wide range of plants in its nursery. The place opens from 7:30 AM to 5:30 PM, their phone number is 98 227 039. Many people recommend the nursery as google gave it a 4.5-star rating.
Nestled in the Beni Khiar, Maison de campagne imed Aounallah is an agricultural hub offering a wide range of services. Located at coordinates (36.4724714, 10.7885366), it operates round-the-clock and specializes in agriculture. With a rating of 5.0, it has received positive feedback from customers. Contact them at 93 573 363 for inquiries.
OLA is a gas station located in Beni Khiar, Tunisia. It offers a variety of fuel options, including gasoline, diesel, and LPG. The gas station also has a convenience store that sells snacks, drinks, and other items. The gas station is open 24 hours a day, 7 days a week. The coordinates of the gas station are (36.4724714, 10.7885366).
"""
result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Given these locations:

  La Pampa is a restaurant located in Beni Khiar, offering a variety of dining options. It has received a 4.4 rating from customers and is open from 7am to 1pm and 5pm to 2am daily. The restaurant's featured image showcases a vibrant and inviting ambiance, and it is located at the coordinates (36.454387640549, 10.785850418423).
Located in Beni Khiar, this bakery has earned a 4.4-star rating from 17 reviews. Its exact location is at FQ5M+V7J, and it can be contacted at 92 422 215 for inquiries or orders.
Cafe des Arcades, located at 15 Av. 14 Janvier 2011, Beni Khiar, is a cafe offering a relaxing atmosphere with a 4.4 rating based on 8 reviews. It operates daily from 4:30 AM to midnight and can be reached via phone at 90 443 842.
LA BAMBA is a Pizzeria located in beni khiar. Its rating is 4.7/5 based on 10 reviews.
Fast Food Haythem is a fast food restaurant located in Beni Khiar, Tunisia. It has a rating of 5.0 based on 4 reviews. It is located at 36.

In [30]:
import pandas as pd

df = pd.read_excel('concatenated_dfs.xlsx')
def process_row(row):
    input_data = row['choices']
    instruction = f"""<start_of_turn>user
Given these locations:\n
{input_data}\n\n
  Generate a good and coherent way to spend the day during a trip.\n"
  Your plan should follow this format:\n
  Day Plan:\n
        Morning: [Activity]\n
        Mid-Morning: [Activity]\n
        Midday: [Activity]\n
        Afternoon: [Activity]\n
        Evening: [Activity]\n
        Night: [Activity]\n
        \n
        Activities can include:\n
        - Exploring [Location]\n
        - Trying local cuisine at [Restaurant]\n
        - Visiting [Landmark]\n
        - Enjoying [Recreational Activity] at [Park/Beach]\n
        - Relaxing with [Activity] at [Spa/Hotel]\n
        - Participating in [Event/Activity] at [Venue]\n
        - Shopping at [Market/Mall]\n
        - Taking a guided tour of [Attraction]\n
        - Engaging in outdoor activities like [Activity] at [Outdoor Location]\n
        - Experiencing cultural immersion at [Museum/Cultural Site]\n
        - Enjoying scenic views at [Scenic Spot]\n
        \n
        Feel free to customize the activities based on your preferences and interests.
  <end_of_turn>\n<start_of_turn>model"""
    output = f"{row['plan']}"
    return {'instruction': instruction, 'input': input_data, 'output': output}

In [31]:
processed_data = df.apply(process_row, axis=1).tolist()
dataset = []
for data_point in processed_data:
    instruction = data_point['instruction']
    input_data = data_point['input']
    output = data_point['output']
    dataset.append({'instruction': instruction, 'input': input_data, 'output': output})
dataset_df = pd.DataFrame(dataset)
dataset_df.to_csv('dataset.csv', index=False)
dataset_df.head()

,instruction,input,output
0,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe La Sirene is a ca...,Day Plan:\nMorning: Exploring Koussour Gharnat...
1,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCAFE LAADHARI is a caf...,Day Plan:\nMorning: Explore the Municipality o...
2,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe Hssine Khlaf is a...,Day Plan:\nMorning: Exploring the historic Mun...
3,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafé République (Sala7...,Day Plan:\nMorning: Explore the charming Café ...
4,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe Lynda is a cafe l...,Day Plan:\nMorning: Explore the Municipality o...


In [32]:
final_dataset = []
for data_point in dataset:
    text = data_point['instruction']
    final_dataset.append({
        "text" : text,
        "instruction": data_point["instruction"],
        "input": data_point["input"],
        "output": data_point["output"],
        "prompt": data_point["instruction"],
    })
final_dataset = pd.DataFrame(final_dataset)
final_dataset.head()

,text,instruction,input,output,prompt
0,<start_of_turn>user\nGiven these locations:\n\...,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe La Sirene is a ca...,Day Plan:\nMorning: Exploring Koussour Gharnat...,<start_of_turn>user\nGiven these locations:\n\...
1,<start_of_turn>user\nGiven these locations:\n\...,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCAFE LAADHARI is a caf...,Day Plan:\nMorning: Explore the Municipality o...,<start_of_turn>user\nGiven these locations:\n\...
2,<start_of_turn>user\nGiven these locations:\n\...,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe Hssine Khlaf is a...,Day Plan:\nMorning: Exploring the historic Mun...,<start_of_turn>user\nGiven these locations:\n\...
3,<start_of_turn>user\nGiven these locations:\n\...,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafé République (Sala7...,Day Plan:\nMorning: Explore the charming Café ...,<start_of_turn>user\nGiven these locations:\n\...
4,<start_of_turn>user\nGiven these locations:\n\...,<start_of_turn>user\nGiven these locations:\n\...,Given these locations:\nCafe Lynda is a cafe l...,Day Plan:\nMorning: Explore the Municipality o...,<start_of_turn>user\nGiven these locations:\n\...


We'll need to tokenize our data so the model can understand.


In [33]:
from datasets import Dataset
final_dataset = Dataset.from_pandas(final_dataset)
final_dataset = final_dataset.shuffle()
final_dataset = final_dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/5502 [00:00<?, ? examples/s]

In [35]:
final_dataset = final_dataset.train_test_split(test_size=0.1)
train_data = final_dataset["train"]
test_data = final_dataset["test"]

Split dataset into 90% for training and 10% for testing

In [36]:
print(train_data)


Dataset({
    features: ['text', 'instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 4951
})


In [37]:
print(test_data)

Dataset({
    features: ['text', 'instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 551
})


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [38]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [39]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [40]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [41]:
modules = find_all_linear_names(model)
print(modules)

['down_proj', 'k_proj', 'v_proj', 'up_proj', 'gate_proj', 'o_proj', 'q_proj']


In [42]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [43]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [44]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=500,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4951 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.544400
2,2.631400
3,2.060600
4,1.815300
5,1.946600
6,1.904800
7,1.846400
8,1.694300
9,1.848300
10,1.691500


In [ ]:
new_model = "planned.AI"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

## Test out Finetuned Model

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model
  

   a Python program to code the Fibonacci series using recursion. Here's a solution:

```python
def fibonacci(n):
  """Calculates the nth Fibonacci number using recursion.

  Args:
    n: The index of the Fibonacci number to calculate.

  Returns:
    The nth Fibonacci number.
  """

  # Base case: The first two Fibonacci numbers are 0 and 1.
  if n <= 1:
    return n

  # Recursive case: Otherwise, calculate the nth Fibonacci number by adding the previous two numbers.
  else:
    return fibonacci(n-1) + fibonacci(n-2)


# Print the Fibonacci numbers.
for i in range(10):
  print(fibonacci(i))
```

**Explanation:**

* The `fibonacci` function takes an integer `n` as input.
* If `n` is less than or equal to 1, it returns `n` itself, as the base case.
* Otherwise, it calculates `n`-th Fibonacci number recurs